In [3]:
import sqlite3


conn = sqlite3.connect('result/places.db')
cursor = conn.cursor()

copy_table_block = """
    DROP TABLE IF EXISTS places;
    ATTACH DATABASE 'result/placev2.db' AS enhanced;
    CREATE TABLE places AS SELECT * FROM enhanced.places;
    DETACH DATABASE enhanced;
"""
cursor.executescript(copy_table_block)
conn.commit()
conn.close()
# copy from enhanced_google_places.db
# attach enhanced..
# copy
## detach

In [4]:
import sqlite3

conn = sqlite3.connect('result/places.db')
cursor = conn.cursor()

# Lấy schema của bảng places
cursor.execute("PRAGMA table_info(places);")
result = cursor.fetchall()
for col in result:
    print(col)

conn.close()

(0, 'address', 'TEXT', 0, None, 0)
(1, 'categories_0', 'TEXT', 0, None, 0)
(2, 'categoryname', 'TEXT', 0, None, 0)
(3, 'cuisine_tags', 'TEXT', 0, None, 0)
(4, 'imageurl', 'TEXT', 0, None, 0)
(5, 'imageurls_0', 'TEXT', 0, None, 0)
(6, 'location_lat', 'REAL', 0, None, 0)
(7, 'location_lng', 'REAL', 0, None, 0)
(8, 'phone', 'TEXT', 0, None, 0)
(9, 'primary_cuisine', 'TEXT', 0, None, 0)
(10, 'rank', 'INT', 0, None, 0)
(11, 'reviews_0_islocalguide', 'REAL', 0, None, 0)
(12, 'reviews_0_likescount', 'REAL', 0, None, 0)
(13, 'reviews_0_name', 'TEXT', 0, None, 0)
(14, 'reviews_0_originallanguage', 'TEXT', 0, None, 0)
(15, 'reviews_0_publishat', 'TEXT', 0, None, 0)
(16, 'reviews_0_publishedatdate', 'TEXT', 0, None, 0)
(17, 'reviews_0_reviewerid', 'TEXT', 0, None, 0)
(18, 'reviews_0_reviewernumberofreviews', 'REAL', 0, None, 0)
(19, 'reviews_0_reviewerphotourl', 'TEXT', 0, None, 0)
(20, 'reviews_0_reviewerurl', 'TEXT', 0, None, 0)
(21, 'reviews_0_reviewid', 'TEXT', 0, None, 0)
(22, 'reviews_0_rev

In [ ]:
import sqlite3

conn = sqlite3.connect('result/places.db')
cursor = conn.cursor()

cursor.execute()
result = cursor.fetchall()
for col in result:
    print(col)

conn.close()

New column order:
- title : TEXT
- address : TEXT
- categories_0 : TEXT
- categoryname : TEXT
- cuisine_tags : TEXT
- imageurl : TEXT
- imageurls_0 : TEXT
- location_lat : REAL
- location_lng : REAL
- phone : TEXT
- primary_cuisine : TEXT
- rank : INT
- reviews_0_islocalguide : REAL
- reviews_0_likescount : REAL
- reviews_0_name : TEXT
- reviews_0_originallanguage : TEXT
- reviews_0_publishat : TEXT
- reviews_0_publishedatdate : TEXT
- reviews_0_reviewerid : TEXT
- reviews_0_reviewernumberofreviews : REAL
- reviews_0_reviewerphotourl : TEXT
- reviews_0_reviewerurl : TEXT
- reviews_0_reviewid : TEXT
- reviews_0_reviewimageurls_0 : TEXT
- reviews_0_reviewimageurls_1 : TEXT
- reviews_0_revieworigin : TEXT
- reviews_0_reviewurl : TEXT
- reviews_0_stars : REAL
- reviews_0_text : TEXT
- reviews_0_texttranslated : TEXT
- reviews_0_translatedlanguage : TEXT
- reviews_1_islocalguide : REAL
- reviews_1_likescount : REAL
- reviews_1_name : TEXT
- reviews_1_originallanguage : TEXT
- reviews_1_publ

In [5]:
import sqlite3

db_path = "result/places.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# ---config--
REMOVE_COLUMNS = {
    "categories_0", 
    "categoryname", 
    "cuisine_tags", 
    "primary_cuisine",
    "rank",
}

RENAME_MAP = {
    "totalscore": "rating",
    "url": "mapURL",
}

PRIORITY_COLUMNS = [
    "title",
    "address",
    "location_lat",
    "location_lng",
    "website",
    "opening_hours",
    "imageurl",
    "phone",
    "rating",  
    "mapURL", 
]

cursor.execute("PRAGMA table_info(places);")
cols_info = cursor.fetchall()

col_names = [c[1] for c in cols_info]
col_types = {c[1]: c[2] for c in cols_info}

remaining_old_cols = [
    name for name in col_names 
    if name not in REMOVE_COLUMNS
]

def new_name(old_name: str) -> str:
    return RENAME_MAP.get(old_name, old_name)

remaining_new_cols = [new_name(name) for name in remaining_old_cols]


new_order = []


for col in PRIORITY_COLUMNS:
    if col in remaining_new_cols and col not in new_order:
        new_order.append(col)

for new_col in remaining_new_cols:
    if new_col not in new_order:
        new_order.append(new_col)

new_col_types = {}
for old_name in remaining_old_cols:
    t = col_types[old_name] or ""
    new_col_types[new_name(old_name)] = t

# 6. Build phần CREATE TABLE
col_defs_sql = ",\n    ".join(
    f'"{name}" {new_col_types[name]}'.strip()
    for name in new_order
)

select_exprs = []
for new_col in new_order:
    old_candidates = [
        old for old in remaining_old_cols 
        if new_name(old) == new_col
    ]
    if not old_candidates:
        raise RuntimeError(f"column '{new_col}' not found")
    old_col = old_candidates[0]
    select_exprs.append(f'"{old_col}"')

create_table_sql = f"""
BEGIN TRANSACTION;

CREATE TABLE places_new (
    {col_defs_sql}
);

INSERT INTO places_new ({", ".join(f'"{name}"' for name in new_order)})
SELECT {", ".join(select_exprs)}
FROM places;

DROP TABLE places;

ALTER TABLE places_new RENAME TO places;

COMMIT;
"""

cursor.executescript(create_table_sql)
conn.commit()
conn.close()


In [6]:
import sqlite3

db_path = "result/places.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute("SELECT * FROM places LIMIT 1")
row = cursor.fetchone()

col_names = [desc[0] for desc in cursor.description]

for name, value in zip(col_names, row):
    print(f"{name}: {value}")

conn.close()


title: agataJapan
address: 19 Đặng Dung, Phường Tân Định, Quận 1, Thành phố Hồ Chí Minh, Vietnam
location_lat: 10.7924419
location_lng: 106.6907725
website: http://starmark.co.jp/
imageurl: https://lh3.googleusercontent.com/gps-cs-s/AG0ilSzMtPXeE1uEkq2fRV1rpRaBW5MZMN-caZKmFtpRkjdvwC2LdEHP6xHOcwxsBEr2mObNWrCh5pFkq9InYJFr50TAUlfVGJS1P62hnKukoXOCmCglpwUz3skQhOcDLKx5H-JsW3Gq=w408-h306-k-no
phone: +84 979 301 670
rating: 4.7
mapURL: https://www.google.com/maps/search/?api=1&query=agataJapan&query_place_id=ChIJLa5tqs0odTERqDsie8VR9u0
imageurls_0: https://lh3.googleusercontent.com/gps-cs-s/AG0ilSzMtPXeE1uEkq2fRV1rpRaBW5MZMN-caZKmFtpRkjdvwC2LdEHP6xHOcwxsBEr2mObNWrCh5pFkq9InYJFr50TAUlfVGJS1P62hnKukoXOCmCglpwUz3skQhOcDLKx5H-JsW3Gq=w1920-h1080-k-no
reviews_0_islocalguide: 0.0
reviews_0_likescount: 0.0
reviews_0_name: Lien Phan
reviews_0_originallanguage: vi
reviews_0_publishat: 2 years ago
reviews_0_publishedatdate: 2023-03-23T15:36:29.787Z
reviews_0_reviewerid: 113861286625110792765
reviews_0_re

In [7]:
import sqlite3
import json

db = "result/places.db"
conn = sqlite3.connect(db)
cursor = conn.cursor()

cursor.execute("PRAGMA table_info(places);")
cols = cursor.fetchall()
col_names = [c[1] for c in cols]

review_groups = {0:[], 1:[], 2:[]}

for name in col_names:
    if name.startswith("reviews_"):
        parts = name.split("_")
        idx = int(parts[1])
        review_groups[idx].append(name)

## add json column reviews
cursor.execute("ALTER TABLE places ADD COLUMN reviews TEXT;")
conn.commit()

cursor.execute("SELECT rowid, * FROM places")
rows = cursor.fetchall()

col_index_map = {name: i+1 for i, name in enumerate(col_names)}   # +1 because rowid at index 0

for row in rows:
    rowid = row[0]
    reviews_json = []

    for idx in review_groups:
        review_obj = {}
        for col in review_groups[idx]:
            clean_key = col.replace(f"reviews_{idx}_", "")
            review_obj[clean_key] = row[col_index_map[col]]
        reviews_json.append(review_obj)

    json_text = json.dumps(reviews_json, ensure_ascii=False)

    cursor.execute("UPDATE places SET reviews = ? WHERE rowid = ?", (json_text, rowid))

conn.commit()

print("STEP 3 done: all reviews converted to JSON column.")


STEP 3 done: all reviews converted to JSON column.


In [8]:
import sqlite3

db_path = "result/places.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute("PRAGMA table_info(places);")
cols_info = cursor.fetchall()

# cols_info: (cid, name, type, notnull, dflt_value, pk)
col_names = [c[1] for c in cols_info]
col_types = {c[1]: c[2] for c in cols_info}

REMOVE_COLUMNS = {
    name for name in col_names
    if ("review" in name.lower()) and (name != "reviews")
}

print("would remove:")
for name in sorted(REMOVE_COLUMNS):
    print("-", name)

remaining_cols = [name for name in col_names if name not in REMOVE_COLUMNS]

col_defs_sql = ",\n    ".join(
    f'"{name}" {col_types[name] or ""}'.strip()
    for name in remaining_cols
)

create_table_sql = f"""
BEGIN TRANSACTION;

CREATE TABLE places_new (
    {col_defs_sql}
);

INSERT INTO places_new ({", ".join(f'"{name}"' for name in remaining_cols)})
SELECT {", ".join(f'"{name}"' for name in remaining_cols)}
FROM places;

DROP TABLE places;

ALTER TABLE places_new RENAME TO places;

COMMIT;
"""

cursor.executescript(create_table_sql)
conn.commit()
conn.close()



would remove:
- reviews_0_islocalguide
- reviews_0_likescount
- reviews_0_name
- reviews_0_originallanguage
- reviews_0_publishat
- reviews_0_publishedatdate
- reviews_0_reviewerid
- reviews_0_reviewernumberofreviews
- reviews_0_reviewerphotourl
- reviews_0_reviewerurl
- reviews_0_reviewid
- reviews_0_reviewimageurls_0
- reviews_0_reviewimageurls_1
- reviews_0_revieworigin
- reviews_0_reviewurl
- reviews_0_stars
- reviews_0_text
- reviews_0_texttranslated
- reviews_0_translatedlanguage
- reviews_1_islocalguide
- reviews_1_likescount
- reviews_1_name
- reviews_1_originallanguage
- reviews_1_publishat
- reviews_1_publishedatdate
- reviews_1_reviewerid
- reviews_1_reviewernumberofreviews
- reviews_1_reviewerphotourl
- reviews_1_reviewerurl
- reviews_1_reviewid
- reviews_1_reviewimageurls_0
- reviews_1_reviewimageurls_1
- reviews_1_revieworigin
- reviews_1_reviewurl
- reviews_1_stars
- reviews_1_text
- reviews_1_texttranslated
- reviews_1_translatedlanguage


In [2]:
import pandas as pd

df = pd.read_csv("temp.csv")
KEYWORDS = [
    "title",
    "address",
    "categories",
    "categoryName",
    "imageUrl",
    "imageUrls",
    "location",
    "openingHours",
    "phone",
    "rank",
    "reviews",
    "reviewsTags",
    "totalScore",
    "url",
    "website",
]
## if column.name not contain any word in KEYWORDS, remove it
for col in df.columns:
    if not any(keyword in col for keyword in KEYWORDS):
        df = df.drop(col, axis=1)

df.to_csv("temp.csv", index=False)


In [3]:
import sqlite3
import json

In [5]:
import sqlite3


conn = sqlite3.connect('result/places.db')
cursor = conn.cursor()

cursor.execute("select rowid, imageurls_0, images from places where imageurls_0 like '%streetview%'")
result = cursor.fetchall()
conn.commit()
# conn.close()

print(len(result))
# copy from enhanced_google_places.db
# attach enhanced..
# copy
## detach

47


In [ ]:
for id, imageurls_0, images in result:
    images = json.loads(images)
    assert "streetview" in imageurls_0
    images[0] = imageurls_0
    images_json = json.dumps(images)
    cursor.execute("UPDATE places SET images = ? WHERE rowid = ?", (images_json, id))
# conn.commit()



In [7]:
## read current value of images
cursor.execute("SELECT rowid, images FROM places where images like '%streetview%'")
result = cursor.fetchall()

for id, images in result:
    images = json.loads(images)
    print(images)


['https://streetviewpixels-pa.googleapis.com/v1/thumbnail?panoid=GGMXwKwc-EC4CNkax2SVbA&cb_client=search.gws-prod.gps&w=408&h=240&yaw=28.544186&pitch=0&thumbfov=100']
['https://streetviewpixels-pa.googleapis.com/v1/thumbnail?panoid=RrfpXSoOJ9M-SPsbHYh0vA&cb_client=search.gws-prod.gps&w=408&h=240&yaw=341.15222&pitch=0&thumbfov=100']
['https://streetviewpixels-pa.googleapis.com/v1/thumbnail?panoid=1mz15ENSJHvqd9KvZyiAsQ&cb_client=search.gws-prod.gps&w=408&h=240&yaw=156.61194&pitch=0&thumbfov=100']
['https://streetviewpixels-pa.googleapis.com/v1/thumbnail?panoid=xSTHSPQOFsnfsWChouLGxw&cb_client=search.gws-prod.gps&w=408&h=240&yaw=285.04922&pitch=0&thumbfov=100']
['https://streetviewpixels-pa.googleapis.com/v1/thumbnail?panoid=WSrCjrOsv1QI5ARoxDS2bg&cb_client=search.gws-prod.gps&w=408&h=240&yaw=309.56137&pitch=0&thumbfov=100']
['https://streetviewpixels-pa.googleapis.com/v1/thumbnail?panoid=dPeUbfB9-OZLJDa3dRV-8w&cb_client=search.gws-prod.gps&w=408&h=240&yaw=15.084415&pitch=0&thumbfov=100'

In [8]:
conn.commit()
conn.close()